# STOCK CLOSING PRICE PREDICTION

In [1]:
#importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import pandas_datareader as pdr

key="30c0973b8a648106ae38faea9031b9a3924c7469"

In [2]:
#Getting Dataset
df=pdr.get_data_tiingo('DIS', api_key=key)

C:\Users\sneha\AppData\Local\Temp\ipykernel_23016\3742500115.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df=pdr.get_data_tiingo('DIS', api_key=key)


In [3]:
df.to_csv('DIS.csv')

In [4]:
df=pd.read_csv('DIS.csv', date_parser=True)

In [5]:
# df.head()

In [6]:
# df.tail()

## Data Pre-processing

In [7]:
df['date'] = pd.to_datetime(df['date'])
# df['date'] = df['date'].dt.date
df

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,DIS,2018-12-10 00:00:00+00:00,111.86,112.290,109.530,111.99,7337457,110.508864,110.933670,108.207008,110.637294,7337457,0.0,1.0
1,DIS,2018-12-11 00:00:00+00:00,111.97,113.720,111.430,112.97,6861759,110.617535,112.346397,110.084058,111.605456,6861759,0.0,1.0
2,DIS,2018-12-12 00:00:00+00:00,112.21,113.980,112.150,113.00,7723872,110.854636,112.603257,110.795361,111.635094,7723872,0.0,1.0
3,DIS,2018-12-13 00:00:00+00:00,113.39,113.600,112.340,112.87,6661534,112.020383,112.227847,110.983066,111.506664,6661534,0.0,1.0
4,DIS,2018-12-14 00:00:00+00:00,112.20,113.920,112.050,112.37,7602301,110.844757,112.543982,110.696569,111.012704,7602301,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,DIS,2023-11-30 00:00:00+00:00,92.69,93.860,92.200,93.55,13396699,92.690000,93.860000,92.200000,93.550000,13396699,0.0,1.0
1253,DIS,2023-12-01 00:00:00+00:00,92.58,92.840,91.770,92.74,11898169,92.580000,92.840000,91.770000,92.740000,11898169,0.0,1.0
1254,DIS,2023-12-04 00:00:00+00:00,92.01,92.410,91.610,91.90,12602805,92.010000,92.410000,91.610000,91.900000,12602805,0.0,1.0
1255,DIS,2023-12-05 00:00:00+00:00,90.79,91.780,89.931,91.61,12356988,90.790000,91.780000,89.931000,91.610000,12356988,0.0,1.0


In [8]:
# Splitting train and test set
df_training=df[df['date']<='2022-01-01'].copy()
df_test=df[df['date']>'2022-01-01'].copy()
df_test, df_training

(     symbol                      date   close     high       low    open  \
 772     DIS 2022-01-03 00:00:00+00:00  156.76  157.560  155.3600  155.83   
 773     DIS 2022-01-04 00:00:00+00:00  155.73  160.320  155.5500  158.59   
 774     DIS 2022-01-05 00:00:00+00:00  155.19  159.380  155.1000  156.52   
 775     DIS 2022-01-06 00:00:00+00:00  156.90  157.770  153.6791  156.24   
 776     DIS 2022-01-07 00:00:00+00:00  157.83  159.300  156.2900  156.90   
 ...     ...                       ...     ...      ...       ...     ...   
 1252    DIS 2023-11-30 00:00:00+00:00   92.69   93.860   92.2000   93.55   
 1253    DIS 2023-12-01 00:00:00+00:00   92.58   92.840   91.7700   92.74   
 1254    DIS 2023-12-04 00:00:00+00:00   92.01   92.410   91.6100   91.90   
 1255    DIS 2023-12-05 00:00:00+00:00   90.79   91.780   89.9310   91.61   
 1256    DIS 2023-12-06 00:00:00+00:00   91.50   92.545   90.6200   91.11   
 
         volume  adjClose  adjHigh    adjLow  adjOpen  adjVolume  divCash 

In [9]:
training_df=df_training.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
training_df

,close,high,low,open,volume
0,111.86,112.29,109.530,111.99,7337457
1,111.97,113.72,111.430,112.97,6861759
2,112.21,113.98,112.150,113.00,7723872
3,113.39,113.60,112.340,112.87,6661534
4,112.20,113.92,112.050,112.37,7602301
...,...,...,...,...,...
767,152.80,154.02,151.400,152.88,7765470
768,155.20,156.50,152.410,152.62,12198679
769,154.87,155.92,154.180,154.56,8737004
770,155.93,157.07,155.515,155.71,7228438


## Feature Scaling

In [10]:
scaler=MinMaxScaler(feature_range=(0,1))
training_df=scaler.fit_transform(training_df)
training_df



array([[0.22470943, 0.21608778, 0.26184131, 0.23769394, 0.05451065],
       [0.22565648, 0.22844306, 0.27817416, 0.24616448, 0.0488937 ],
       [0.22772277, 0.23068948, 0.28436345, 0.24642379, 0.05907337],
       ...,
       [0.59500646, 0.5930534 , 0.6456632 , 0.60564415, 0.07103623],
       [0.60413259, 0.60298946, 0.65713917, 0.61558408, 0.05322338],
       [0.59517865, 0.59866943, 0.65047709, 0.61696703, 0.04096006]])

## Converting into Dataset Matrix (taking previous 60 values)

In [11]:
X_train=[]
y_train=[]

In [12]:
training_df.shape

(772, 5)

In [13]:
ltrain=training_df.shape[0]

for i in range(60, ltrain):
    X_train.append(training_df[i-60:i])
    y_train.append(training_df[i,0])


In [14]:
X_train, y_train=np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((712, 60, 5), (712,))

## Building LSTM 

In [15]:
regressior=Sequential()
regressior.add(LSTM(units=60, activation='relu', return_sequences=True, input_shape=(X_train.shape[1],5)))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=60, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=80, activation='relu', return_sequences=True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=120, activation='relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units=1))

regressior.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 60)            15840     
                                                                 
 dropout (Dropout)           (None, 60, 60)            0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 60)            29040     
                                                                 
 dropout_1 (Dropout)         (None, 60, 60)            0         
                                                                 
 lstm_2 (LSTM)               (None, 60, 80)            45120     
                                                                 
 dropout_2 (Dropout)         (None, 60, 80)            0         
                                                                 
 lstm_3 (LSTM)               (None, 120)               9

In [16]:
regressior.compile(optimizer='adam', loss='mean_squared_error')
regressior.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
12/12 [==============================] - 21s 353ms/step - loss: 0.1449
Epoch 2/50
12/12 [==============================] - 6s 460ms/step - loss: 0.0180
Epoch 3/50
12/12 [==============================] - 6s 495ms/step - loss: 0.0152
Epoch 4/50
12/12 [==============================] - 5s 412ms/step - loss: 0.0115
Epoch 5/50
12/12 [==============================] - 5s 436ms/step - loss: 0.0100
Epoch 6/50
12/12 [==============================] - 4s 291ms/step - loss: 0.0099
Epoch 7/50
12/12 [==============================] - 3s 288ms/step - loss: 0.0102
Epoch 8/50
12/12 [==============================] - 4s 352ms/step - loss: 0.0099
Epoch 9/50
12/12 [==============================] - 4s 320ms/step - loss: 0.0087
Epoch 10/50
12/12 [==============================] - 4s 326ms/step - loss: 0.0080
Epoch 11/50
12/12 [==============================] - 3s 280ms/step - loss: 0.0074
Epoch 12/50
12/12 [==============================] - 4s 295ms/step - loss: 0.0079
Epoch 13/50
12/12 [=====

## Preparing Test Dataset

In [17]:
df_test.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
772,DIS,2022-01-03 00:00:00+00:00,156.76,157.56,155.3600,155.83,10228257,156.76,157.56,155.3600,155.83,10228257,0.0,1.0
773,DIS,2022-01-04 00:00:00+00:00,155.73,160.32,155.5500,158.59,16582041,155.73,160.32,155.5500,158.59,16582041,0.0,1.0
774,DIS,2022-01-05 00:00:00+00:00,155.19,159.38,155.1000,156.52,12272097,155.19,159.38,155.1000,156.52,12272097,0.0,1.0
775,DIS,2022-01-06 00:00:00+00:00,156.90,157.77,153.6791,156.24,11095326,156.90,157.77,153.6791,156.24,11095326,0.0,1.0
776,DIS,2022-01-07 00:00:00+00:00,157.83,159.30,156.2900,156.90,9554622,157.83,159.30,156.2900,156.90,9554622,0.0,1.0


In [18]:
df_training.tail()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
767,DIS,2021-12-27 00:00:00+00:00,152.80,154.02,151.400,152.88,7765470,152.80,154.02,151.400,152.88,7765470,0.0,1.0
768,DIS,2021-12-28 00:00:00+00:00,155.20,156.50,152.410,152.62,12198679,155.20,156.50,152.410,152.62,12198679,0.0,1.0
769,DIS,2021-12-29 00:00:00+00:00,154.87,155.92,154.180,154.56,8737004,154.87,155.92,154.180,154.56,8737004,0.0,1.0
770,DIS,2021-12-30 00:00:00+00:00,155.93,157.07,155.515,155.71,7228438,155.93,157.07,155.515,155.71,7228438,0.0,1.0
771,DIS,2021-12-31 00:00:00+00:00,154.89,156.57,154.740,155.87,6189861,154.89,156.57,154.740,155.87,6189861,0.0,1.0


In [19]:
past_60_days=df_training.tail(60)
# past_60_days
test_df=past_60_days.append(df_test) #past 60 days taken for the first value of test data
test_df=test_df.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
test_df

C:\Users\sneha\AppData\Local\Temp\ipykernel_23016\458701125.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df=past_60_days.append(df_test) #past 60 days taken for the first value of test data


,close,high,low,open,volume
712,177.71,179.630,177.3300,177.70,7779997
713,176.74,178.600,176.4029,177.93,4132182
714,173.52,176.520,173.4800,176.02,8046270
715,173.13,174.490,172.5450,173.62,5188380
716,172.96,173.700,170.9400,173.63,8918152
...,...,...,...,...,...
1252,92.69,93.860,92.2000,93.55,13396699
1253,92.58,92.840,91.7700,92.74,11898169
1254,92.01,92.410,91.6100,91.90,12602805
1255,90.79,91.780,89.9310,91.61,12356988


In [ ]:
# test_df_scaled = scaler.transform(test_df)

In [20]:
# scaler=MinMaxScaler(feature_range=(0,1))
test_df=scaler.fit_transform(test_df)
test_df


array([[1.        , 1.        , 1.        , 0.99005924, 0.05515183],
       [0.99014127, 0.98968308, 0.99059728, 0.99236871, 0.        ],
       [0.95741437, 0.96884891, 0.96095299, 0.97319008, 0.05917765],
       ...,
       [0.12897652, 0.12636724, 0.13062087, 0.12852696, 0.12806854],
       [0.11657689, 0.12005689, 0.11359231, 0.12561502, 0.124352  ],
       [0.12379307, 0.12771946, 0.12058021, 0.12059444, 0.09455624]])

In [21]:
test_df.shape

(545, 5)

In [22]:
X_test=[]
y_test=[]

In [23]:
ltest=test_df.shape[0]
for i in range(60,ltest):
    X_test.append(test_df[i-60:i])
    y_test.append(test_df[i,0])

In [24]:
X_test, y_test=np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((485, 60, 5), (485,))

## Predicting the stock closing price

In [25]:
y_pred=regressior.predict(X_test)

16/16 [==============================] - 3s 86ms/step


In [26]:
import math
from sklearn.metrics import mean_squared_error

math.sqrt(mean_squared_error(y_test, y_pred))

0.06198227035168991

In [27]:
y_test

array([7.87071857e-01, 7.76603313e-01, 7.71114951e-01, 7.88494766e-01,
       7.97946946e-01, 7.85445675e-01, 7.98556764e-01, 7.97642037e-01,
       7.73655859e-01, 7.38083139e-01, 7.41437138e-01, 7.19483687e-01,
       6.94176237e-01, 5.90100620e-01, 5.90913711e-01, 5.81258258e-01,
       5.51682082e-01, 5.70179896e-01, 6.02805163e-01, 6.46915337e-01,
       6.62364061e-01, 6.43358065e-01, 6.17034251e-01, 6.37259884e-01,
       6.42240065e-01, 6.41935156e-01, 6.90212420e-01, 7.40319138e-01,
       7.12978961e-01, 7.27004777e-01, 7.66338042e-01, 7.82904767e-01,
       7.48348409e-01, 7.32188231e-01, 6.98953146e-01, 6.76389877e-01,
       7.12267507e-01, 7.13588779e-01, 7.02713690e-01, 6.74662059e-01,
       6.91330420e-01, 6.73340787e-01, 6.24047159e-01, 5.50665718e-01,
       5.32879358e-01, 5.52190263e-01, 5.52088627e-01, 5.32879358e-01,
       5.05234272e-01, 5.57780262e-01, 5.97824982e-01, 6.11342616e-01,
       6.19778433e-01, 6.02703527e-01, 6.17847342e-01, 5.92743165e-01,
      

## Visualization the results

In [34]:
import plotly.graph_objs as go

# Create a trace for actual values
trace_actual = go.Scatter(x=df_test['date'], y=y_test, mode='lines', name='Actual', line=dict(color='red'))

# Create a trace for predicted values
trace_pred = go.Scatter(x=df_test['date'], y=y_pred.flatten(), mode='lines', name='Predicted', line=dict(color='green'))

# Create a trace for markers with reduced size
trace_markers1 = go.Scatter(
    x=df_test['date'],
    y=y_test.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='red', size=3),  # Adjust the size here
)

trace_markers2 = go.Scatter(
    x=df_test['date'],
    y=y_pred.flatten(),
    mode='markers',
    name='',
    showlegend=False,
    marker=dict(color='green', size=3),  # Adjust the size here
)

# Create the Plotly figure
fig = go.Figure(data=[trace_actual, trace_pred, trace_markers1, trace_markers2])

# Customize the layout
fig.update_layout(
    title='Stock Price Prediction',
    xaxis_title='Time',
    yaxis_title='Stock Price',
     width=800,
    height=600
)

# Show the Plotly figure
fig.show()


In [29]:
# # Generate predictions for the next 10 days
# last_60_days = df1[-60:]  # Take the last 60 days from the scaled data
# X_next_10_days = []  # Initialize the input sequence for the next 10 days

# for i in range(10):
#     X_next_10_days.append(last_60_days[-60:])
#     last_60_days = np.concatenate([last_60_days[1:], y_pred[-1].reshape(1, 1)])

# X_next_10_days = np.array(X_next_10_days)

# # Reshape the input data for prediction
# X_next_10_days = np.reshape(X_next_10_days, (X_next_10_days.shape[0], X_next_10_days.shape[1], 5))

# # Predict the next 10 days
# y_next_10_days = regressior.predict(X_next_10_days)

# # Invert the scaling to get the predicted stock prices in the original scale
# y_next_10_days = scaler.inverse_transform(np.concatenate([last_60_days[-10:].reshape(-1, 1), y_next_10_days]))

# # Create a new DataFrame for the next 10 days with the predicted values and the corresponding dates
# next_10_days_dates = pd.date_range(start=df_test['date'].iloc[-1] + pd.Timedelta(days=1), periods=10)
# df_next_10_days = pd.DataFrame({'date': next_10_days_dates, 'predicted_price': y_next_10_days.flatten()})

# # Print the predicted stock prices for the next 10 days
# print(df_next_10_days)


In [30]:
# df_past_60_days_for_future = df_test.tail(60).copy()
# df_past_60_days_for_future=df_past_60_days_for_future.drop(['symbol','date','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor'], axis=1)
# df_past_60_days_for_future

# X_input=list(df_past_60_days_for_future)
# X_input = df_past_60_days_for_future.iloc[:, 0].tolist()
# X_input

# # demonstrate prediction for next 10 days
# from numpy import array
# import pandas as pd 

# lst_output=[]
# n_steps=60
# i=0

# while i < 10:
#     if len(X_input) > 60:
#         X_input = np.array(temp_input[1:])
#         print("{} day input {}".format(i, X_input))
#         X_input = X_input.reshape(1, n_steps, 1)
#         yhat = regressior.predict(X_input, verbose=0)
#         print("{} day output {}".format(i, yhat))
#         temp_input.extend(yhat[0].tolist())
#         temp_input = temp_input[1:]
#         lst_output.extend(yhat.tolist())
#         i = i + 1
#     else:
#         X_input = np.array(X_input).reshape((1, n_steps, 1))
#         yhat = regressior.predict(X_input, verbose=0)
#         print("{} day output {}".format(i, yhat))
#         temp_input.extend(yhat[0].tolist())
#         lst_output.extend(yhat.tolist())
#         i = i + 1

# print(lst_output)

# # day_new=np.arange(1,61)
# # day_pred=np.arange(61,71)
# # plt.plot(day_new,df_train['Close'],'b',label='Actual')
# # plt.plot(day_pred,lst_output,'r',label='Predicted')
# # plt.legend()
# # plt.show()




In [31]:
# X_train.shape[1]
# print(len(temp_input))


In [32]:
# df_past_60_days_for_future